In [2]:
import gymnasium as gym
from gymnasium.spaces import Box, Discrete,Tuple
import numpy as np
import pygame

# Define colors
WHITE = (255, 255, 255)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)  # Color for the trajectory

class CustomEnv(gym.Env):
    metadata = {"render_modes": ["human", "rgb_array"], "render_fps": 60}

    def __init__(self,render_mode=None):
        super().__init__()
        self.grid_size = 10
        self.pause = False
        self.domain_randomization = False
        self.render_mode = render_mode

        # Define two separate thresholds for obstacle handling
        self.distance_threshold_penalty = 5  # Penalty zone threshold (larger value)
        self.distance_threshold_collision = 1.5  # Collision threshold (smaller value)
        self.distance_threshold_arm = 3  # Arm threshold (smaller value)
        self.penalty_factor = 5  # Penalty scaling factor
        self.distance_reward_factor = 2
        self.smooth_action_penalty = 2
        self.steps = 0
        self.margin = 0.3
        self.reward_arm = -100
        self.reward_hand = -100
        self.reward_bound = -200
        self.reward_max_step = 200
        self.reward_step = 10
        self.stride_robot_random = [1,3]
        self.stride_hand_random = [0.6,1]
        self.hand_move_epsilon = 0.1


        self.current_distance = 0  # Current distance to goal, used for reward shaping
        self.max_steps = 50  # Set a maximum number of steps to prevent infinite loops
        # Action space (dx, dy)
        self.action_space = Box(low=-1, high=1, shape=(2,), dtype=np.float32)
        # Observation space (robot_x, robot_y, goal_x, goal_y)
        self.observation_shape = 2+2+2+1+1+1+2+1+1 # Robot position, hand position, velocity_hand,radius_hand, and distance to hand

        self.observation_space = Box(low=0, high=np.array([self.grid_size*2,self.grid_size, self.grid_size*2, self.grid_size,1,1 , (2**0.5)*self.grid_size,0.5*self.grid_size,0.5*self.grid_size,2*self.grid_size,self.grid_size,self.stride_robot_random[1],self.stride_hand_random[1]]), 
                                     shape=(self.observation_shape,), dtype=np.float32)

        self.random = True
        # For rendering
        self.window = None
        self.clock = None
        self.cell_size = 50 # Pixels per grid unit
        self.trajectory_points = [] # New: List to store past robot positions
        self.dist_arm = 0


    def dist_point_to_segment_correct(self,P, A, B, eps=1e-12):
        P = np.asarray(P, dtype=float)
        A = np.asarray(A, dtype=float)
        B = np.asarray(B, dtype=float)
        v = B - A
        w = P - A
        vv = np.dot(v, v)
        if vv <= eps:
            # A and B coincide: treat as point A
            C = A.copy()
            d = np.linalg.norm(P - A)
            t = 0.0
            case = 'endpoint_A'
        else:
            t = np.dot(w, v) / vv
            if t < 0.0:
                C = A
                d = np.linalg.norm(P - A)
                case = 'before_A'
            elif t > 1.0:
                C = B
                d = np.linalg.norm(P - B)
                case = 'after_B'
            else:
                C = A + t * v
                d = np.linalg.norm(P - C)
                case = 'on_segment'
        return float(d), C, float(t), case

    def _get_obs(self):
        
        return np.concatenate(([self.robot_position]+ [self.hand_position]+[self.last_action]+
                               [np.array([self.current_distance])]+
                               [np.array([min(self.robot_position[0],
                                              self.robot_position[1],
                                              self.grid_size-self.robot_position[0],
                                              self.grid_size-self.robot_position[1]) ])]+
                                              [np.array([self.dist_arm])]+
                                               [self.fixed_point]+[np.array([self.stride_robot])]+[np.array([self.stride_hand])]))

    def _get_info(self):
        return {
            "distance_to_hand": self.current_distance,
            "robot_position": self.robot_position,
            "hand_position": self.hand_position,
            'distance_arm':self.dist_arm,
            "fix_point":self.fixed_point,
        }

    def reset(self, seed=None, options=None):

        super().reset()
        self.distance = []
        self.stride_robot = np.random.uniform(*self.stride_robot_random)  # Randomize stride length
        self.stride_hand = np.random.uniform(*self.stride_hand_random)  # Randomize stride length
        # self.stride_robot = 1  # Randomize stride length
        self.distance_threshold_collision = np.random.uniform(2,3)  # Randomize collision threshold
        self.distance_threshold_penalty = np.random.uniform(3, 4)  # Randomize penalty threshold
        
        
        self.noise_obs_sigma = np.random.uniform(0, 0.1)  # Add some noise to observation to make it more realistic
        self.noise_action_sigma = np.random.uniform(0,0.1)  # Add some noise to action to make it more realistic
        
        
        
        self.robot_position = np.random.uniform(self.margin, [2*(self.grid_size-self.margin),self.grid_size-self.margin], size=2)
        self.hand_position = np.random.uniform(self.margin, [2*(self.grid_size-self.margin),self.grid_size-self.margin], size=2)
        # self.hand_position = np.clip(self.hand_position, self.margin, self.grid_size-self.margin)  # Ensure hand stays within grid bounds
        
        # self.hand_move_mode = 'random' if np.random.rand() < 0.1 else 'towards_robot'  # Randomize hand movement mode
        # self.hand_move_mode = 'towards_robot'
        
        self.current_distance = np.linalg.norm(self.robot_position - self.hand_position)
        self.pre_distance = self.current_distance
        self.last_action = np.zeros(2)
        self.steps = 0
        self.trajectory_points = [self.robot_position.copy()] # New: Reset trajectory and add initial position
        
        self.fixed_point = np.array([self.grid_size*random.uniform(0.2,1.8),self.grid_size])
        return self._get_obs(), self._get_info()

    def _reward(self,action):
        terminated = False
        truncated = False
        reward = 0  # Initialize reward
        done_reason = None  # Initialize done reason

        # action regulation penalty
        # reward -= 0.5 * np.sum(np.square(action))  # Penalty for large actions

        self.dist_arm = self.dist_point_to_segment_correct(self.robot_position,self.hand_position, self.fixed_point)[0]
        if self.dist_arm < self.distance_threshold_arm:
            reward += self.reward_arm 
            terminated = True  # Truncate if arm is too short

        # boundary penalty
        if np.any(self.robot_position <= self.margin) or (self.grid_size-self.robot_position[1] <=self.margin) or (2*self.grid_size-self.robot_position[0] <=self.margin):
            reward += self.reward_bound
            terminated = True  # Truncate if robot goes out of bounds
            done_reason = "out of bounds"

    
        # Auxiliary Rewards -  distance to hand
        self.current_distance = np.linalg.norm(self.robot_position - self.hand_position)
        self.distance.append(self.current_distance)
        reward += (self.current_distance-self.pre_distance)*self.distance_reward_factor  # Reward shaping based on distance change
        self.pre_distance = self.current_distance

        # Obstacle handling with two thresholds
        if self.current_distance < self.distance_threshold_collision:
            reward += self.reward_hand
            terminated = True  # Terminate if too close to obstacles
            done_reason = "collision with obstacle"
        elif self.current_distance < self.distance_threshold_penalty:
            reward -= self.penalty_factor * (self.distance_threshold_penalty - self.current_distance)  # Penalty for being too close to obstacles

        reward -= self.smooth_action_penalty * np.linalg.norm(action - self.last_action)

        # Small reward for each step taken to encourage exploration
        reward+= self.reward_step 

        # Truncate if max steps reached and give max step reward
        if self.steps >= self.max_steps:
            reward += self.reward_max_step
            truncated = True  

        return reward,terminated,truncated,done_reason

    def _get_hand_movement(self):

        # if self.hand_move_mode == 'random':
        #     move_hand = np.random.uniform(-1, 1, size=2)  # Randomly move the hand position slightly
        # elif self.hand_move_mode == 'towards_robot':
        #     dir_vector = self.robot_position - self.hand_position
        #     if np.linalg.norm(dir_vector) > 0:
        #         dir_vector /= np.linalg.norm(dir_vector)
        #     move_hand = dir_vector * self.stride_hand  # Move hand towards robot position
        if random.random() < self.hand_move_epsilon:
            move_hand = np.random.uniform(-1, 1, size=2)  # Randomly move the hand position slightly
        else:
            dir_vector = self.robot_position - self.hand_position
            if np.linalg.norm(dir_vector) > 0:
                dir_vector /= np.linalg.norm(dir_vector)
            move_hand = dir_vector * self.stride_hand  # Move hand towards robot position
        
        return move_hand






    def step(self, action):
        if self.random:
            action+=np.random.normal(0,self.noise_action_sigma,size=self.action_space.shape)  # Add some noise to action to make it more realistic

        move_hand = self._get_hand_movement()
        self.hand_position += move_hand  # Update hand position
        self.hand_position = np.clip(self.hand_position, self.margin, [self.grid_size*2,self.grid_size-self.margin])  # Ensure hand stays within grid bounds
        # self.fixed_point+= np.array([np.,0])  # Randomize fixed point position

        self.robot_position += action * self.stride_robot  # Scale the action to control speed
        self.trajectory_points.append(self.robot_position.copy()) # New: Add current position to trajectory
        self.steps += 1
        

        reward,terminated,truncated,done_reason = self._reward(action)
        info = self._get_info()
        info['done_reason'] = done_reason
        info['distance_mean'] = np.mean(self.distance)
        observation = self._get_obs()
        if self.random:
            observation += np.random.normal(0, self.noise_obs_sigma, size=self.observation_shape)  # Add some noise to observation to make it more realistic

        return observation, reward, terminated, truncated, info

    def render(self, mode="human"):
 
        pygame.display.init()
        self.window = pygame.display.set_mode(
                (int(self.grid_size * self.cell_size), int(self.grid_size * self.cell_size))
            )
        pygame.display.set_caption("CustomEnv")
        if self.clock is None:
            self.clock = pygame.time.Clock()
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                import sys
                sys.exit() # Exit the program

            elif event.type == pygame.MOUSEBUTTONDOWN:
                mouse_x, mouse_y = event.pos
                self.hand_position = np.array([mouse_x/self.cell_size, mouse_y/self.cell_size])

            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE:  # 空格键切换暂停
                    self.pause = not self.pause


        canvas = pygame.Surface((self.grid_size * self.cell_size, self.grid_size * self.cell_size))
        canvas.fill(WHITE)
        virus_image = pygame.image.load("hand.png").convert_alpha()  # Load an image if needed, but not used here
        robot_image = pygame.transform.scale(virus_image, (int(self.cell_size * 2), int(self.cell_size * 2)))  # Scale the image
        # New: Draw the trajectory
        if len(self.trajectory_points) > 1:
            scaled_points = []
            for point in self.trajectory_points:
                scaled_points.append((int(point[0] * self.cell_size), int(point[1] * self.cell_size)))
            
            # Draw lines between consecutive points
            pygame.draw.lines(canvas, BLUE, False, scaled_points, 2) # Blue line, not closed, 2 pixels wide
            
            # Optionally, draw small circles at each point to emphasize
            for point_coord in scaled_points:
                pygame.draw.circle(canvas, BLUE, point_coord, 3) # Small blue circles

        # Draw robot
        pygame.draw.circle(
            canvas,
            RED,
            (int(self.robot_position[0] * self.cell_size), int(self.robot_position[1] * self.cell_size)),
            int(self.cell_size * 0.2)
        )
        # Draw obstacles

        canvas.blit(robot_image, (int((self.hand_position[0]-1) * self.cell_size), int((self.hand_position[1]-1) * self.cell_size+1)))
        pygame.draw.circle(canvas,
                            GREEN, 
                            (int((self.hand_position[0]) * self.cell_size), 
                            int((self.hand_position[1]) * self.cell_size+1)), 
        int(self.cell_size * 0.2)
        )

        self.window.blit(canvas, canvas.get_rect())
        pygame.event.pump()
        pygame.display.flip()
        self.clock.tick(self.metadata["render_fps"])
        time.sleep(0.5)
    
    def load_args(self, args):
        pass

    def save_args(self,path):
        env_args = {
            "grid_size": self.grid_size,
            "distance_threshold_penalty":self.distance_threshold_penalty,
            "distance_threshold_collision":self.distance_threshold_collision,
            "penalty_factor":self.penalty_factor,
            "distance_reward_factor":self.distance_reward_factor,
            "smooth_action_penalty":self.smooth_action_penalty,
            "max_steps":self.max_steps,
            "margin":self.margin,
            "reward_step":self.reward_step,
            "reward_max_step":self.reward_max_step,
            "reward_bound":self.reward_bound,
            "reward_arm":self.reward_arm,
            "reward_hand":self.reward_hand,
            "stride_robot_range":self.stride_robot_random,
            "stride_hand_range":self.stride_hand_random,
            "move_hand_epsilon":self.hand_move_epsilon,



        }
        with open(os.path.join(path, "env_args.json"), "w") as f:
            json.dump(env_args, f,indent=4)
        

    def close(self):
        pygame.display.quit()
        pygame.quit()





In [3]:
from collections import deque
from stable_baselines3.common.callbacks import BaseCallback, EvalCallback, CallbackList
class DebugCallback(BaseCallback):
    def __init__(self, env, render_freq=10000, n_episodes=1, log_freq=10000, verbose=1):
        super().__init__(verbose)
        self.log_freq = log_freq
        # 用deque统计最近N个done的终止原因，避免内存爆炸
        self.termination_reasons = deque(maxlen=1000)  # 统计最近1000次终止
        self.env_to_render = env
        self.render_freq = render_freq
        self.n_episodes = n_episodes
        self.distance_mean = deque(maxlen=1000) 

    def _on_step(self) -> bool:
        # 先从env info里读取终止原因
        # print((self.locals.keys()))
        infos = self.locals.get('infos', None)

        dones = self.locals.get('dones', None)
        if infos is not None and dones is not None:
            for done, info in zip(dones, infos):
                if done and info is not None and 'done_reason' in info:
                    self.termination_reasons.append(info['done_reason'])
                    self.distance_mean.append(info['distance_mean'])

        # 每log_freq步打印信息
        # if self.num_timesteps % self.render_freq == 0 and self.verbose:
        #     for ep in range(self.n_episodes):
        #         obs = self.env_to_render.reset()
        #         done = False
        #         while not done:
        #             action, _states = self.model.predict(obs, deterministic=True)
        #             obs, rewards, done, info = self.env_to_render.step(action)
        #             self.env_to_render.render()
        #             time.sleep(0.6)

        #             if done:

        #                 self.env_to_render.close()
        #                 break





        if self.num_timesteps % self.log_freq == 0 and self.verbose:
            log = self.model.logger.name_to_value
            ep_rew = log.get('rollout/ep_rew_mean', None)
            ep_len = log.get('rollout/ep_len_mean', None)
            loss = log.get('train/loss', None)
            v_loss = log.get('train/value_loss', None)
            p_loss = log.get('train/policy_gradient_loss', None)
            ent_loss = log.get('train/entropy_loss', None)
            kl = log.get('train/approx_kl', None)

            # 统计终止原因比例
            total = len(self.termination_reasons)
            if total > 0:
                count_hand = sum(1 for r in self.termination_reasons if r == 'out of bounds')
                ratio_hand = count_hand / total
            else:
                ratio_hand = 0.0
            distance_mean = sum(self.distance_mean) / len(self.distance_mean) if len(self.distance_mean) > 0 else 0.0

            # print(f"[{self.num_timesteps:7d}] ep_rew_mean={ep_rew}, ep_len_mean={ep_len}, loss={loss:.3f}, "
            #       f"v_loss={v_loss:.3f}, p_loss={p_loss:.3f}, ent_loss={ent_loss:.3f}, kl={kl:.4f}, "
            #       f"termination_reason_hand_ratio={ratio_hand:.3f}")
            self.logger.record("custom/termination_reason_ratio", ratio_hand)

            self.logger.record("custom/distance_mean", distance_mean)
            self.logger.dump(step=self.num_timesteps)

        return True


In [ ]:
from camera_calibration.camera_calibration import CameraCalibration
from robot_control.ur_control import URControl  
from cv.hand_detect import HandDetection
import mediapipe as mp
import cv2
from ultralytics import YOLO
from cv.get_workspace import get_workspace



desired_width = 2592 
desired_height = 1944


cv_model = YOLO('runs/detect/train3/weights/best.onnx')
hand_detector = HandDetection()
cali = CameraCalibration()
robot_ip = "192.168.1.2"

robot_control = URControl(robot_ip)

mp_hands = mp.solutions.hands
mp_draw = mp.solutions.drawing_utils

hands = mp_hands.Hands(
    max_num_hands=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)



cap = cv2.VideoCapture(0)
cv2.namedWindow('Frame', cv2.WINDOW_NORMAL)  # 创建一个窗口来显示矫正后的图像
cv2.namedWindow('edges', cv2.WINDOW_NORMAL)  
if not cap.isOpened():
    print("Error: Could not open camera for demonstration.")
    exit()
cap.set(cv2.CAP_PROP_FRAME_WIDTH, desired_width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, desired_height)

w_env, h_env = 20, 10  # 环境的宽度和高度



In [ ]:
from dataclasses import dataclass


@dataclass
class BufferRecord:
    obs: list
    action: list
    reward: list
    next_obs: list
    done: list

    def __len__(self):
        return len(self.obs)

    def add(self, obs, action, reward, next_obs, done):
        self.obs.extend(obs)
        self.action.extend(action)
        self.reward.extend(reward)
        self.next_obs.extend(next_obs)
        self.done.extend(done)


    def to_dict(self):
        return {
            "obs": self.obs,
            "action": self.action,
            "reward": self.reward,
            "next_obs": self.next_obs,
            "done": self.done
        }
    def save_to_npz(self, file_path):
        np.savez(file_path, **self.to_dict())
    
    def load_from_npz(self, file_path):
        data = np.load(file_path)
        self.obs = data["obs"]
        self.action = data["action"]
        self.reward = data["reward"]
        self.next_obs = data["next_obs"]
        self.done = data["done"]


In [ ]:
import gymnasium as gym
import torch
import os
from stable_baselines3 import PPO,SAC
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize, VecMonitor
from stable_baselines3.common.monitor import Monitor
# from stable_baselines3.commom.buffers import ReplayBuffer
from stable_baselines3.common.logger import configure
from stable_baselines3.common.utils import safe_mean

import random
import json
import time

env_raw = CustomEnv()
save_path = "logs/best_model_sac"
i=1
while os.path.exists(save_path+str(i)):
    i+=1
save_path = f"{save_path}{i}"
os.makedirs(save_path)

tensorboard_dir = "./sac_custom_env_tensorboard"
os.makedirs(tensorboard_dir, exist_ok=True)
tensorboard_log_dir = tensorboard_dir + '/'+ f'best_model_sac{i}'
load_model = "../../RL/src/logs/best_model_sac59/best_model.zip"





# env = Monitor(env_raw)  # Wrap the environment with Monitor for logging
env = env_raw

# env = VecNormalize(env, norm_obs=True, norm_reward=True)
policy_kwargs = dict(
    net_arch=[dict(pi=[128,256,256,128], qf=[128,256,256,128])],
    # activation_fn=torch.nn.ReLU  # 改为 ReLU，通常更适合稀疏奖励
)
policy_kwargs = dict(
    net_arch=[128,256,256,128],
    # activation_fn=torch.nn.ReLU  # 改为 ReLU，通常更适合稀疏奖励
)
# model = PPO("MlpPolicy", env, verbose=1,policy_kwargs=policy_kwargs,tensorboard_log="./ppo_custom_env_tensorboard/")
# model = PPO.load("logs/best_model4/best_model.zip",env=env)  # Load the best model
# model.learning_rate = 0.0008
# model.ent_coef = 0.02  # Set a lower learning rate for fine-tuning


model = SAC("MlpPolicy", env, verbose=1,ent_coef='auto',buffer_size=10_000,policy_kwargs=policy_kwargs,tensorboard_log=tensorboard_log_dir)

model = SAC.load(load_model, env, verbose=1,ent_coef='auto',learning_rate=0.0001,tensorboard_log=tensorboard_log_dir)
replay_buffer = model.replay_buffer

eval_callback = EvalCallback(
    env,
    best_model_save_path=save_path,
    log_path = './logs/',
    eval_freq=10,  # 每1000步评估一次
    deterministic=True,
    render=True,
    n_eval_episodes=10,  # 每次评估5个episode
)

last_hand = 0
position_hand_env = [0,0]
last_action = np.array([0,0],dtype=np.float32)
h,w =0,0
def get_obs(frame):
    global last_hand, position_hand_env,last_action,h,w,position_robot_pixel
    # ret, frame = cap.read()
    # if not ret:
    #     print("Error: Could not read frame from camera for demonstration.")
    # 使用 cv2.undistort 对每一帧进行畸变矫正

    undistorted_frame = cali.undistort_frame(frame)
    undistorted_frame = get_workspace(undistorted_frame)
    h , w = undistorted_frame.shape[:2]
    img_gray = cv2.cvtColor(undistorted_frame, cv2.COLOR_BGR2GRAY)
    results = cv_model.predict(undistorted_frame, conf=0.7, save=False,imgsz=640,verbose=False)
    undistorted_frame, hand_positions = hand_detector.process_frame(undistorted_frame)
    for i, r in enumerate(results):
        boxes = r.boxes
        for box in boxes:
            # 提取边界框坐标

            x1, y1, x2, y2 = map(int, box.xyxy[0])
            if x2-x1 > 100:
                continue
            # 计算中心点
            cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
            # 在图像上绘制边界框和中心点
            cv2.rectangle(undistorted_frame, (x1, y1), (x2, y2), (255, 0, 255), 6)
            # cv2.circle(undistorted_frame, (cx, cy), 5, (255, 0, 0), -1)
    
    if hand_positions:
        position_hand_env = hand_positions[0]/np.array([w/h_env,h/w_env])
        position_hand_env = position_hand_env[1],h_env - position_hand_env[0]
    
    hsv = cv2.cvtColor(undistorted_frame, cv2.COLOR_BGR2HSV)

    # 设置肤色范围（适用于常见肤色，可根据光照微调）
    lower_skin = np.array([0, 30, 60], dtype=np.uint8)
    upper_skin = np.array([20, 150, 255], dtype=np.uint8)

    # 根据阈值生成mask
    mask = cv2.inRange(hsv, lower_skin, upper_skin)

    ys, xs = np.where(mask > 0)
    # 例如右边伸出
    try:
        idx = np.argmin(xs)
        tip = (xs[idx], ys[idx])
        cv2.circle(undistorted_frame, tip, 10, (0, 0, 255), -1)
        fixed_point = [tip[1]*20/h,10]
    except:
        fixed_point = [10,10]
    
    *position_robot_world,z,rx,ry,rz = robot_control.get_robot_pose()  # 使用更新后的类名





    position_robot_pixel = cali.world_to_pixel(position_robot_world)
    print("实际位置")
    print(position_robot_world)
    
    position_robot_env =  position_robot_pixel[1]*w_env / h, h_env - position_robot_pixel[0]*h_env /w 





    robot = np.array([position_robot_env],dtype=np.float32)
    hand = np.array([position_hand_env],dtype=np.float32)
    stride_hand = np.linalg.norm(hand - last_hand)
    # print(stride_hand)
    distance_to_object = np.linalg.norm(robot - hand)
    distance = np.array([distance_to_object],dtype=np.float32)
    # print(distance)
    boundary = np.array([min(position_robot_env[0],position_robot_env[1],10-position_robot_env[0],10-position_robot_env[1])],dtype=np.float32)
    # last_action = np.array([last_action],dtype=np.float32)


    dist_arm = env_raw.dist_point_to_segment_correct(robot.flatten(),hand.flatten(),[15,10])[0]
    # fixed_point = [tip[1]*20/h,10]
    stride_robot = 0.5
    env.hand_position = hand.flatten()
    env.robot_position = robot.flatten()
    env.fixed_point = fixed_point
    last_hand = hand
    last_action = action
    obs = np.concatenate((robot.flatten(),hand.flatten(),last_action.flatten(),distance.flatten(),boundary.flatten(),np.array([dist_arm],dtype=np.float32),np.array(fixed_point,dtype=np.float32),np.array([stride_robot]),np.array([stride_hand])))

    return obs

debug_callback = DebugCallback(env=env,log_freq=10000, verbose=1)
# debug_callback = None
callback = CallbackList([eval_callback, debug_callback])

# model._logger = configure("./logs/manual_sac_train/", ["stdout", "tensorboard"])
# model._current_progress_remaining = 1.0

model._setup_learn(total_timesteps=400000,callback=callback)
callback.on_training_start(locals(), globals())
model.learn(total_timesteps=400000,callback=callback)
obs = env.reset()
done = False
rewards = []
best_eval_metric = 0
info_buffer = deque(maxlen=100)
for step in range(1000000):
    model.num_timesteps+=1


    # get obs
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read frame from camera for demonstration.")
        break
    

    obs = get_obs(frame)








    action, _ = model.predict(obs, deterministic=True)
    # print(f"action: {action}")
    # next_obs, reward, done, info = env.step(action)


    #execute action
    stride_robot = env.stride_robot
    action_pixel = action * np.array([h/w_env,w/h_env])*stride_robot
    action_pixel = -action_pixel[1],action_pixel[0]
    rx,ry,rz = 0.256,-0.229,-4.984
    position_robot_pixel += np.array([action_pixel[0],action_pixel[1]])
    position_robot_world = cali.pixel_to_world(position_robot_pixel)
    print("目标位置")
    print(position_robot_world)
    robot_control.move_robot([position_robot_world[0],position_robot_world[1],0.125,rx,ry,rz],0)
    time.sleep(0.5)


    # compute reward,done,info

    reward,terminated,truncated,done_reason = env._reward(action)
    done = terminated or truncated
    info= [{"terminated":terminated,"truncated":truncated,"done_reason":done_reason}]

    


    # get next obs
    ret, frame = cap.read()
    next_obs = get_obs(frame)
    # print(next_obs[:2])

    # store transition in replay buffer
    replay_buffer.add(obs,next_obs, action, reward, done, info)
    rewards.append(reward)

    if step % 50 == 0 and replay_buffer.size() > 64:
        time.sleep(0.5)
        print("training")
        model.train(batch_size=64, gradient_steps=16)

    
    if done:
        print("*************done*************")
        # time.sleep(1)
        model._episode_num += 1
        info_buffer.append({
            "r": safe_mean(rewards),
            "l": len(rewards),
        })
        # print(f"Episode {model._episode_num} mean reward: {safe_mean(rewards)}")
        # print(f"Episode {model._episode_num} episode length: {len(rewards)}")
        rewards = []

        if model._episode_num % 8 == 0:
            # model.dump_logs()
            model.logger.record("rollout/ep_rew_mean", safe_mean([ep_info["r"] for ep_info in info_buffer]))
            model.logger.record("rollout/ep_len_mean", safe_mean([ep_info["l"] for ep_info in info_buffer]))
            model.logger.dump(step=model.num_timesteps)
        # obs = env.reset()
        done = False
    # callback.on_step()
callback.on_training_end()
# env.save("vec_env.pkl")  # Save the environment state

settings = {
    'load_model':load_model,
    'tensorboard_log' :tensorboard_log_dir,
           }
env_raw.save_args(save_path)
with open(os.path.join(save_path, "settings.json"), "w") as f:
    json.dump(settings, f)

env.close()

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to ./sac_custom_env_tensorboard/best_model_sac85\run_1


c:\Users\admin\anaconda3\envs\RL\lib\site-packages\gymnasium\spaces\box.py:306: UserWarning: WARN: Box high's precision lowered by casting to float32, current high.dtype=float64
  gym.logger.warn(


实际位置
[-0.12711553076978113, -1.037458889582425]
目标位置
[    -0.1203    -0.99267]
实际位置
[-0.12036782918994725, -0.9926260394636907]
实际位置
[-0.12036758453671047, -0.9926340411900599]
目标位置
[   -0.12933    -0.97435]
实际位置
[-0.12925769214654012, -0.9743520150472583]
实际位置
[-0.1293431335771027, -0.9743248556334682]
目标位置
[      -0.12     -1.0097]
实际位置
[-0.12011702053843937, -1.0097402539309048]
实际位置
[-0.12008157986428175, -1.0097460676127281]
目标位置
[   -0.09039      -1.042]
实际位置
[-0.09044349288661316, -1.042004306505304]
实际位置
[-0.0904147201860152, -1.0419975857136854]
目标位置
[  -0.055628     -1.0023]
实际位置
[-0.055695208912310466, -1.002243664725373]
实际位置
[-0.05569994033031819, -1.0022733168579825]
目标位置
[   -0.03264    -0.95229]
实际位置
[-0.032686669543565264, -0.9522587856651135]
实际位置
[-0.03266210339854674, -0.9522441413397833]
目标位置
[  -0.027335    -0.89788]
实际位置
[-0.02747641234350594, -0.8978171288197571]
实际位置
[-0.027400218203573322, -0.897864842747235]
目标位置
[  -0.049951    -0.92368]


KeyboardInterrupt: 